In [ ]:
!pip install openai langchain pydub
from google.colab import userdata
import os,random

from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.callbacks import StreamlitCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)

import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
OPENAI_API_MODEL = 'gpt-4'
OPENAI_API_TEMPERATURE = 0.5

topic = "European medieval history"

#PlaygroundからView Codeしてそのまま貼り付ける
##########################################################################
playground_viewcode=[
  {
    "role": "system",
    "content": "As a English teacher who experienced in teaching Japanese students and knowledgeable about their English proficiency levels, create a script of about 250 words, related to the topic of " + topic + ". This script should be suitable for Part 2 of the Listening section of the Eiken Test, Grade 1, a widely recognized English proficiency test in Japan. After the script, include two related questions. The script's subject matter should cover areas including, but not limited to science, nature, society, history, and politics, incorporating vocabulary appropriate for Eiken Test Grade 1. Title the script succinctly, using no more than four words. Present the content in a clear, direct manner, without an opening greeting or a lecture-style format. \n\n\n\n\n\n"
  },
  {
    "role": "user",
    "content": "Please generate the instructed content in system.  The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {\"properties\": {\"foo\": {\"title\": \"Foo\", \"description\": \"a list of strings\", \"type\": \"array\", \"items\": {\"type\": \"string\"}}}, \"required\": [\"foo\"]}\nthe object {\"foo\": [\"bar\", \"baz\"]} is a well-formatted instance of the schema. The object {\"properties\": {\"foo\": [\"bar\", \"baz\"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{\"properties\": {\"passage_text\": {\"title\": \"Passage Text\", \"type\": \"string\"}, \"questions\": {\"title\": \"Questions\", \"type\": \"array\", \"items\": {\"$ref\": \"#/definitions/Question\"}}}, \"required\": [\"passage_text\", \"questions\"], \"definitions\": {\"Question\": {\"title\": \"Question\", \"type\": \"object\", \"properties\": {\"question_text\": {\"title\": \"Question Text\", \"type\": \"string\"}, \"options\": {\"title\": \"Options\", \"type\": \"array\", \"items\": {\"type\": \"string\"}}, \"correct_answer\": {\"title\": \"Correct Answer\", \"type\": \"string\"}, \"explanation\": {\"title\": \"Explanation\", \"type\": \"string\"}}, \"required\": [\"question_text\", \"options\", \"correct_answer\", \"explanation\"]}}}\n```"
  },
  {
    "role": "assistant",
    "content": "{\n   \"passage_text\": \"Sleep and Dementia. Researchers in France have been studying possible causes of dementia, a condition in which a person experiences a severe loss of cognitive function. They investigated whether the amount of sleep a person gets in middle age plays a role in the development of the condition. Using self-reported sleep data from a large-scale lifestyle survey, the researchers looked at 8,000 participants who had been research subjects since 1985. The team found that, compared to participants who reported sleeping seven hours per night in their 50s and 60s, those who regularly got six hours of sleep or less per night were 30% more likely to develop dementia in later decades. While this finding suggests that insufficient sleep could be a contributing factor in the development of dementia, it is not conclusive. It could be that lack of sleep in middle age is actually an early symptom of dementia, rather than a cause. However, a neurologist who commented on the study thinks this is probably not the case, as the poor sleeping habits arose such a long time before any of the subjects were diagnosed with dementia. Additionally, she points out that the first biological change that leads to dementia — the buildup of certain proteins in the brain — typically does not begin that early on\",\n   \"questions\": [\n       {\n           \"question_text\": \"What is one thing the researchers in France examined?\",\n           \"options\": [\"1 Evidence that shows people with dementia sleep more.\", \"2 Various sleep disorders reported by young people.\", \"3 Lifestyle changes as a treatment for dementia.\", \"4 The sleep patterns of middle-aged people.\"],\n           \"correct_answer\": \"4\",\n           \"explanation\": \"第1段落前半に、フランスの研究者たちが中年期の睡眠時間が認知症の発症にどのような役割を果たすかを調査したとあります。そして、50代と60代で一晩に7時間睡眠した参加者と比べて、6時間以下の睡眠をとっていた参加者は後の年代で認知症を発症するリスクが30％高かったという結果が示されています。これに基づいて、正解は「4. 中年期の人々の睡眠パターン」です。\"\n       },\n       {\n           \"question_text\": \"What does the neurologist believe?\",\n           \"options\": [\"1 Protein buildup is not always a sign of dementia.\", \"2 Participants' sleep patterns were likely not caused by dementia.\", \"3 Better treatments for dementia will soon be developed.\", \"4 Middle-aged people generally have difficulty sleeping.\"],\n           \"correct_answer\": \"2\",\n           \"explanation\": \"第2段落に、ある神経学者が中年期の睡眠不足が実際には認知症の初期症状ではないと考えていると述べられています。これは、睡眠不足の習慣が認知症の診断よりもずっと前に始まっていたこと、そして認知症につながる脳内の特定のタンパク質の蓄積がそれほど早く始まることは通常ないという点に基づいています。そのため、正解は「2. 参加者の睡眠パターンはおそらく認知症によるものではない」となります。\"\n       }\n   ]\n}\n"
  },
  {
    "role": "assistant",
    "content": "{\n   \"passage_text\": \"American Camels. Today, camels are an iconic symbol of the Middle East and North Africa. Surprisingly, however, evidence indicates that they originated in North America. Fossils suggest that the last indigenous camels in North America died out many thousands of years ago. Then, in the mid-1800s, the US government imported several dozen camels to deliver military supplies to remote areas. It was thought that the animals would be well-suited to the harsh desert climate of the southwestern United States. Indeed, initial field exercises were so successful that a high-ranking official suggested acquiring an additional thousand camels. When the American Civil War broke out shortly after the camels had been put to work, however, the plan was abandoned. Some of the camels that had been brought over were sold to private businesses such as circuses and mining operations for their value as entertainment and labor, but a few were released and continued living in the desert. Although there were several wild-camel sightings over the following decades, their numbers were too few to create a stable population, and it is thought they disappeared completely. Now, over 100 years later, some scientists are considering reintroducing camels a second time.\",\n   \"questions\": [\n       {\n           \"question_text\": \"Why did the US government import camels in the mid-1800s?\",\n           \"options\": [\"1 To supply goods to isolated regions.\", \"2 To improve relationships with foreign countries.\", \"3 To restore the ecology of desert areas.\", \"4 To give them as gifts to senior military officers.\"],\n           \"correct_answer\": \"1\",\n           \"explanation\": \"第1段落中ほどに、米国政府がラクダを輸入したのは、遠隔地域への軍事物資の輸送のためだったと書かれています。このことから、選択肢1「孤立した地域への物資の供給」が正解です。\"\n       },\n       {\n           \"question_text\": \"What do we learn about the camels that were released into the wild?\",\n           \"options\": [\"1 They were captured to be sold to circuses.\", \"2 They eventually spread across the desert.\", \"3 Their population was too small to sustain itself.\", \"4 They continued to be sighted for nearly 100 years.\"],\n           \"correct_answer\": \"3\",\n           \"explanation\": \"第2段落の最後に、野生に放されたラクダは、安定した集団を形成するには数が少なすぎたと述べられています。そのため、選択肢3「彼らの集団は自己維持するには小さすぎた」が正解です。\"\n       }\n   ]\n}\n"
  }
]
##########################################################################

# OpenAIのPlayground, view Codeからコピーしたmessage部分をLangchainベースに変換する関数を定義
def format_messages(original_messages):
    formatted_messages = []
    for msg in original_messages:
        if msg['role'] == 'system':
            formatted_messages.append(SystemMessage(content=msg['content']))
        elif msg['role'] == 'user':
            formatted_messages.append(HumanMessage(content=msg['content']))
        elif msg['role'] == 'assistant':
            formatted_messages.append(AIMessage(content=msg['content']))
    return formatted_messages

###########################################################################
#LangChainのオブジェクトに変換できたことを確認
messages = format_messages(playground_viewcode)



#pydanticを使用してクラスを作成、アウトプットをパースできるようにしておく
from typing import List, Dict
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

class Question(BaseModel):
    question_text: str
    options: List[str]  # 文字列のリストとして定義
    correct_answer: str
    explanation: str

class ExamQuestion(BaseModel):
    passage_text: str
    questions: List[Question]  # Question クラスのインスタンスのリスト

# # 使用例
# exam_question = ExamQuestion(
#     passage_text="Antivenins. Every year, millions of people are bitten by venomous snakes. Although treatments known as antivenins exist, lives that could be saved are being lost due to supply problems and substandard products. In developing countries, snakebite victims who cannot afford proper treatments often receive partial doses or cheap antivenins that are ineffective for the snakes that bit them. To make matters worse, when these low-grade antivenin treatments are unsuccessful, victims turn instead to traditional healers. As a result, antivenin sales fall, causing some manufacturers to stop producing them altogether. Currently, antivenins are made by injecting snake venom into animals so that the animals produce proteins called antibodies, which are then extracted. These negate the effects of the venom. However, such products often have serious side effects. Furthermore, they have to be stored at low temperatures, meaning that victims must often travel for hours to hospitals, during which time their symptoms worsen. However, a new antivenin is being developed that may solve these problems. It is an artificial substance that does not need to be refrigerated. As a result, it could be kept on hand and administered immediately after a snakebite, greatly improving victims chances of survival.",
#     questions=[
#         Question(
#             question_text="What is one problem for snakebite victims in developing countries?",
#             options=["1 It is difficult to identify what kind of snake bit them.", "2 Traditional medicines are generally unavailable.", "3 They sometimes receive poor-quality antivenins.", "4 Doctors often give them too much antivenin."],  # 正しい形式のリスト
#             correct_answer="3",
#             explanation="第1段落中ほどで、発展途上国では、proper treatments を受ける余裕のない被害者は、partial doses であったり、かんできた蛇には効かない cheap antivenins を投与されることが多いと述べられている。3では、この問題点を時々poor-quality antiveninsを投与されると言い換えている。"
#         ),
#         Question(
#             question_text="What is one thing we learn about the new antivenin?",
#             options=["1 It is more expensive than other antivenins. ", "2 Its production causes harm to animals. ", "3 It is only effective for mild snakebites. ", "4 It can be kept at room temperature."],  # 正しい形式のリスト
#             correct_answer="4",
#             explanation="新たな蛇毒血清に関する情報は第 2段落後半で語られている。そこには、冷蔵しておく必要のないan artificial substance なので、手に置いておいて、かまれた後すぐに投与できるようになる、とある。この冷蔵の必要がないという利点をroom temperature で保管できると言い換えた4が正解。"
#         )
#     ]
# )
#
# print(exam_question)

#Parseができたので、Parsarという変数に書き込んでおく
parser = PydanticOutputParser(pydantic_object=ExamQuestion)

In [ ]:
from openai import OpenAI
# from IPython.display import Audio

chat = ChatOpenAI(model_name=OPENAI_API_MODEL, temperature=OPENAI_API_TEMPERATURE)

output = chat(messages)
exam_question = parser.parse(output.content)

listening_passage_text = exam_question.passage_text
listening_Q1 = exam_question.questions[0].question_text
listening_Q2 = exam_question.questions[1].question_text
listening_input = {"passage": listening_passage_text,
                   "question_1": listening_Q1,
                   "question_2": listening_Q2}


print(exam_question.passage_text+"\n")

In [ ]:
words = ["alloy", "echo", "fable", "nova", "onyx", "shimmer"]
selected = random.sample(words, 2)
name1, name2 = selected[0], selected[1]

client = OpenAI()
new_file_name = 'eiken'

sound_file = client.audio.speech.create(input=listening_passage_text, model="tts-1", voice=name1, speed=1.0)
sound_file.stream_to_file(f"drive/My Drive/passage_{new_file_name}.mp3")
sound_file = client.audio.speech.create(input=listening_Q1, model="tts-1", voice=name2, speed=1.0)
sound_file.stream_to_file(f"drive/My Drive/question1_{new_file_name}.mp3")
sound_file = client.audio.speech.create(input=listening_Q2, model="tts-1", voice=name2, speed=1.0)
sound_file.stream_to_file(f"drive/My Drive/question2_{new_file_name}.mp3")

In [ ]:
print('問１'+"\n")
for items in exam_question.questions[0].options:
  print(items+"\n")
print('問２'+"\n")
for items in exam_question.questions[1].options:
  print(items+"\n")

print("\n"+exam_question.passage_text+"\n")
print("Question 1")
print(exam_question.questions[0].question_text+"\n")
print("Question 2")
print(exam_question.questions[1].question_text+"\n")

print("問１　正解　" + exam_question.questions[0].correct_answer+"\n")
print(exam_question.questions[0].explanation+"\n")
print("問２　正解　" + exam_question.questions[1].correct_answer+"\n")
print(exam_question.questions[1].explanation+"\n")

In [ ]:
chat = ChatOpenAI(model_name=OPENAI_API_MODEL, temperature=OPENAI_API_TEMPERATURE)
words_explanation = exam_question.passage_text + 'この文章に含まれている英単語のうち、英検一級に出てくるくらい難しいと考える英単語を列挙してください。この時、単語の文頭の文字は必ず小文字にしてください。それぞれの発音記号、意味、例文、語源、その他覚えやすくなるTipsも同時にお願いします。出力は日本語でお願いします。'
output_words = chat([HumanMessage(content=words_explanation)])
print(output_words.content)